<a href="https://colab.research.google.com/github/abhipraay/CVI_Projects_PS_22_23/blob/main/DL_PS_22_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification Using CNN
#### By The Computer Vision and Intelligence Club (CVI) - IIT Madras



This aim of this Problem Statement is to introduce Deep Learning, which is one of the most used techniques for Computer Vision Application. 
Before beginning go thorugh the following articles:
1. https://www.nature.com/articles/nature14539 : It is fairly easy to follow and provides an excellent overview of the field. (You will need to login with smail to download the pdf.) 

2. http://deeplearning.stanford.edu/tutorial/supervised/MultiLayerNeuralNetworks/ : A short introduction to Multi-layer perceptrons.

3. https://cs231n.github.io/convolutional-networks/ : Introduction to Convolutional Neural Networks. CNNs are generally used for computer vision problems.


This introduction should be sufficient to get you started with this problem statement. If after going through the PS, you guys are interested in further exploring the field, I would suggest the following resources:
1. http://cs231n.stanford.edu/ : Online course by Stanford.
2. http://introtodeeplearning.com/: Video Lectures from MIT
3. https://www.deeplearningbook.org/: Most popular book on Deep Learning


We'll build a CNN using Pytorch to use it to classify thousands of pictures in 6 different categories.

Sounds like a huge task, but do not worry! Remember to use Google freely to search for any doubts, errors or documentation. This assignment is not only to check your DL skills, but also how you can step up to learn new things on the go using publicly available sources.

Ideally at the end, we expect that you understand the code in EVERY cell

Enter your code between areas surrounded by the following comment design
######### YOUR CODE HERE #########


################################

[Link For The Dataset](https://drive.google.com/uc?id=1Qc66kVqetwJIK7cKXnXxbPJy6gnpRSRI)

The aim of this get you all familiarized with Deep Learning in TensorFlow, a very popular Deep Learning Library (or in general GPU computation library).

Some of the dataloading and preprocessing work has been done, and you guys are expected to fill-in code where you are asked to. 

Before diving into the code ensure that you copy the notebook to your drive (See the option in File Tab) and that the Runtime Type is set to GPU (Runtime tab -> Change runtime type). To see the importance of GPU in deep learning see [this](https://course.fast.ai/gpu_tutorial.html) short article.




First we import the necessary libraries. We recommend that you check out what these libraries are if they seem unfamiliar

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
import os
import glob as gb
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader
import torchvision
import time

# Downloading the Data and Extracting it

In [ ]:
!gdown https://drive.google.com/uc?id=1Qc66kVqetwJIK7cKXnXxbPJy6gnpRSRI
!unzip The_Data.zip

Now define the paths to the train test and pred folders

In [ ]:
######### YOUR CODE HERE #########
trainpath = './RandomFolder/RandomSubFolder/TheFolderIWant'
testpath = ''
predpath = ''

##################################

# Data Loading


Now let's first check the Train folder to have a look to its content

In [ ]:
for folder in  os.listdir(trainpath + 'seg_train') : 
    files = gb.glob(pathname= str( trainpath +'seg_train//' + folder + '/*.jpg'))
    print(f'For training data , found {len(files)} in folder {folder}')

Ok, how about the test folder

In [ ]:
for folder in  os.listdir(testpath +'seg_test') : 
    files = gb.glob(pathname= str( testpath +'seg_test//' + folder + '/*.jpg'))
    print(f'For testing data , found {len(files)} in folder {folder}')

_____
Now for prediction folder

In [ ]:
files = gb.glob(pathname= str(predpath +'seg_pred/*.jpg'))
print(f'For Prediction data , found {len(files)}')

_____

# Checking Images

Now we need to check the images sizes , to know how they look like

Since we have 6 categories , we first need to create a dictionary with their names & indices. This is known as integer encoding. Also create a function to get the code back

In [ ]:
code_to_num = {'buildings':0 ,'forest':1, 'glacier':2, 'mountain':3, 'sea':4, 'street':5}
num_to_code = {0:'buildings' ,1:'forest', 2:'glacier', 3:'mountain', 4:'sea', 5:'street'}

def get_code(n) : 
    if n in num_to_code:
        return num_to_code[n]    

def get_num(c):
    if c in code_to_num:
        return code_to_num[c] 

Now how about the images sizes in train folder

In [ ]:
size = []
for folder in  os.listdir(trainpath +'seg_train') : 
    files = gb.glob(pathname= str( trainpath +'seg_train//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

______

Ok, almost all of them are (150,150,3), how about test images ? 

In [ ]:
size = []
for folder in  os.listdir(testpath +'seg_test') : 
    files = gb.glob(pathname= str( testpath +'seg_test//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts()

Almost same ratios  
Now to prediction images 

In [ ]:
size = []
files = gb.glob(pathname= str(predpath +'seg_pred/*.jpg'))
for file in files: 
    image = plt.imread(file)
    size.append(image.shape)
pd.Series(size).value_counts()

Ok , since almost all of pictures are (150,150,3) , we can use all pictures in our model, after resizing it to a particular size

# Reading Images

Now it's time to read all images & convert it into arrays

First we'll create a variable s , which refer to size , so we can change it easily 

Let's use now size = 100 , so it will be suitable amount to contain accuracy without losing so much time in training

In [ ]:
s = 100

Now to read all pictues in six categories in training folder, and use OpenCV to resize it. And not to forget assigning the y value from the predefined function 

In [ ]:
from PIL import Image
X_train_image = []   # contains the image with data type unit8 for visualisation
X_train = []         # converted to tensor and normalised for training
y_train = []
for folder in  os.listdir(trainpath +'seg_train') : 
    files = gb.glob(pathname= str( trainpath +'seg_train//' + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        image = (torch.tensor(np.array(cv2.resize(image,(s,s)))).permute(2,0,1) )/255
        
        X_train_image.append(image_array)
        X_train.append(image)
        y_train.append(get_num(folder))

Great , now how many items in X_train 

In [ ]:
print(f'we have {len(X_train)} items in X_train')

Also we have have a look to random pictures in X_train , and to adjust their title using the y value

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_train_image[i])   
    plt.axis('off')
    plt.title(get_code(y_train[i]))

Great , now to repeat same steps exactly in test data

In [ ]:
X_test_image = []
X_test = []
y_test = []
for folder in  os.listdir(testpath +'seg_test') : 
    files = gb.glob(pathname= str(testpath + 'seg_test//' + folder + '/*.jpg'))
    for file in files: 
        image = cv2.imread(file)
        image_array = cv2.resize(image , (s,s))
        image = (torch.tensor(np.array(cv2.resize(image,(128,128)))).permute(2,0,1) )/255
        X_test.append(image)         
        X_test_image.append(list(image_array))
        y_test.append(get_num(folder))
        

In [ ]:
print(f'we have {len(X_test)} items in X_test')

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_test_image),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_test_image[i])    
    plt.axis('off')
    plt.title(get_code(y_test[i]))

Also with Prediction data , without having title ofcourse

In [ ]:
X_pred_image = []
X_pred = []
files = gb.glob(pathname= str(predpath + 'seg_pred/*.jpg'))
for file in files: 
    image = cv2.imread(file)
    image_array = cv2.resize(image , (s,s))
    image = (torch.tensor(np.array(cv2.resize(image,(128,128)))).permute(2,0,1) )/255
    X_pred.append(image)
    X_pred_image.append(list(image_array))    

In [ ]:
print(f'we have {len(X_pred)} items in X_pred')

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred_image),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred_image[i])    
    plt.axis('off')

________

# Building The Model 

Now we need to build the model to train our data

converting the tensors to dataloaders

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
# first make a trainset and then the trainloader

In [ ]:
# first make a testset and then the testloader

In [ ]:
# first make a predset and then the predloader

Now to build the CNN model using pytorch, using Conv2D layers , MaxPooling & Dropouts and Dense layer

In [ ]:
############### YOUR CODE HERE  ####################
class Model(nn.Module):
      def __init__(self):
        super(Model, self).__init__()  

      def forward(self, x): # x is the input image

        return 

####################################################

So how the model looks like ? 

In [ ]:
model = Model()
print(model)

Now to train the model , lets use 50 epochs now

In [ ]:
############### YOUR CODE HERE  ####################

epochs = 50
learning_rate = ?

criterion = ?


optimizer = ?

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct/total

def train(model, dataset, optimizer, criterion,epoch):

    model.train()
    train_loss=[]
    train_acc=[]
    for batch_idx,(data,target) in enumerate(dataset):
      
        output=model(data)
        optimizer.zero_grad()
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
        acc = accuracy(output, target)
        train_acc.append(acc)
        train_loss.append(loss.item())
        print('\repoch:{}({:.0f}%)\tloss:{:.3f}\ttrain_accuracy:{:.2f}%'.format(epoch+1,100*batch_idx/len(dataset),
        np.mean(train_loss),100*np.mean(train_acc)),end='')
    print()

def eval(model, dataset, criterion):
    
    model.eval()
    val_acc=[]
    for batch_idx,(data,target) in enumerate(dataset):
        output=model(data)
        acc = accuracy(output, target)
        val_acc.append(acc)
    print('val_accuracy:{:.2f}%'.format(100*np.mean(val_acc))) 
    return np.mean(val_acc)



for epoch in range(epochs):
      start_time = time.monotonic()
      train(model, trainloader, optimizer, criterion,epoch)
      eval_accuracy = eval(model, testloader, criterion)
      end_time = time.monotonic()
      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      print("TIME TAKEN FOR THE EPOCH: {} mins and {} seconds\n".format(epoch_mins, epoch_secs))
print("OVERALL TRAINING COMPLETE")      


####################################################

How is the final loss & accuracy?


In [ ]:
eval(model, testloader, criterion)


_______

Now to predict X_test

In [ ]:
y_pred = []
for  data, target in testloader:
  data, target = data.to(device), target.to(device)
  output = model(data.double())
  _, preds = torch.max(output.data, 1)
  preds = preds.cpu().numpy()
  y_pred.append(preds)
y_pred = np.array(y_pred)
y_pred = np.concatenate( y_pred, axis=0 )
print('Prediction Shape is {}'.format(y_pred.shape))


Now it's time to predict X_Predict

In [ ]:
y_result = []
for data in predloader:
  data = data.to(device)
  output = model(data.double())
  _, preds = torch.max(output.data, 1)
  preds = preds.cpu().numpy()
  y_result.append(preds)
y_result = np.array(y_result)
y_result = np.concatenate( y_result, axis=0 )  
print('Prediction Shape is {}'.format(y_result.shape))

And show random redicted pictures & its predicting category


In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')
    plt.title(get_code(np.argmax(y_result[i])))

In [ ]:
#### BONUS POINTS FOR SOME TRANSFER LEARNING APPROACH

[Basics Of Transfer Learning](https://machinelearningmastery.com/transfer-learning-for-deep-learning/)